In [ ]:
# ==============================================================
# 0. LIBRERÍAS Y CONFIGURACIÓN INICIAL
# ==============================================================

import os
import random
import math

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# --------------------------------------------------------------
# Intentamos importar MLflow (si no está instalado, el código
# sigue funcionando pero sin registrar experimentos)
# --------------------------------------------------------------
try:
    import mlflow
    import mlflow.keras as mlflow_keras
    MLFLOW_AVAILABLE = True
except ImportError:
    MLFLOW_AVAILABLE = False
    print("Aviso: MLflow no está instalado. "
          "El código correrá igual, pero sin registrar experimentos.")

# --------------------------------------------------------------
# Fijamos semillas para reproducibilidad básica
# --------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)


# ==============================================================
# 1. CARGA DEL CONJUNTO DE DATOS LIMPIO
# ==============================================================

# Ruta al archivo limpio (ajusta el nombre si es necesario)
DATA_PATH = "listings_model_no_outliers.csv"

# Leemos el CSV en un DataFrame
df = pd.read_csv(DATA_PATH)

# Mostramos información básica para verificar que cargó bien
print("Shape del DataFrame:", df.shape)
print("Primeras columnas:", df.columns.tolist()[:20])


# ==============================================================
# 2. DEFINICIÓN DE LA VARIABLE OBJETIVO (REGRESIÓN)
# ==============================================================

# En el proyecto queremos predecir el "precio por noche".
# Buscamos el nombre de la columna de target.
posibles_targets = ["price", "price_night", "price_per_night"]
target_col = None

for col in posibles_targets:
    if col in df.columns:
        target_col = col
        break

if target_col is None:
    raise ValueError(
        "No se encontró ninguna columna de precio en el DataFrame. "
        "Revisa el nombre de la columna de precio y actualiza 'posibles_targets'."
    )

print(f"Usaremos la columna '{target_col}' como variable objetivo (y).")

# Si la columna de precio está como texto con símbolos ($, comas), la convertimos a numérico
if df[target_col].dtype == "O":
    df[target_col] = (
        df[target_col]
        .astype(str)
        .str.replace(r"[\$,]", "", regex=True)
        .astype(float)
    )

# Nos aseguramos de que no haya valores nulos en la variable objetivo
df = df[df[target_col].notna()].copy()


# ==============================================================
# 2b. DEFINICIÓN DE ETIQUETA "RECOMENDADO" (PREGUNTA 3)
# ==============================================================

# Esto deja lista la etiqueta 'recomendado' para que tu compañero
# haga el modelo de clasificación (Etapa 4/4b, pregunta 3).

# Nos aseguramos de que existan las columnas necesarias
cols_necesarias = [
    "review_scores_rating",
    "review_scores_cleanliness",
    "number_of_reviews"
]
for c in cols_necesarias:
    if c not in df.columns:
        raise ValueError(
            f"Falta la columna requerida para la regla de recomendación: {c}"
        )

# Creamos la columna 'recomendado' basada en la regla:
# recomendado = 1 si:
#   review_scores_rating       >= 4.0
#   y review_scores_cleanliness >= 4.0
#   y number_of_reviews        >= 5
# recomendado = 0 en caso contrario

mask_valid = (
    df["review_scores_rating"].notna()
    & df["review_scores_cleanliness"].notna()
    & df["number_of_reviews"].notna()
)

df["recomendado"] = 0  # por defecto no recomendado

df.loc[
    mask_valid
    & (df["review_scores_rating"] >= 4.0)
    & (df["review_scores_cleanliness"] >= 4.0)
    & (df["number_of_reviews"] >= 5),
    "recomendado"
] = 1

# Resumen general (para usar en el informe)
total_con_info = mask_valid.sum()
total_recomendados = df.loc[mask_valid, "recomendado"].sum()
porc_recomendados = total_recomendados / total_con_info * 100

print("\n===================================================")
print("CLASIFICACIÓN DE LISTINGS: RECOMENDADOS VS NO RECOMENDADOS")
print("===================================================")
print(f"Listings con info suficiente para aplicar la regla: {total_con_info}")
print(f"Listings recomendados: {total_recomendados} "
      f"({porc_recomendados:.1f}% de los que tienen info suficiente)")
print("La etiqueta 'recomendado' se define por rating >= 4.0, "
      "limpieza >= 4.0 y al menos 5 reseñas.\n")


# ==============================================================
# 3. SELECCIÓN DE VARIABLES PREDICTORAS (X)
# ==============================================================

# a) Eliminamos columnas identificadoras o claramente no útiles para la red
cols_descartar = [
    target_col,
    "id",
    "listing_id",
    "name",
    "description",
    "host_name",
    "host_id",
    "last_review",
    "recomendado",   # esta es para clasificación, no para el precio
]

cols_descartar = [c for c in cols_descartar if c in df.columns]

# Tomamos todas las demás columnas como candidatos
X_raw = df.drop(columns=cols_descartar)

# b) Convertimos columnas booleanas a 0/1 y luego nos quedamos
#    con todas las columnas numéricas (incluye dummies one-hot)
bool_cols = X_raw.select_dtypes(include=["bool"]).columns
print("Columnas booleanas encontradas (se convierten a 0/1):", len(bool_cols))

if len(bool_cols) > 0:
    X_raw[bool_cols] = X_raw[bool_cols].astype(int)

X_num = X_raw.select_dtypes(include=[np.number]).copy()

print("Variables numéricas iniciales:", X_num.shape[1])

# c) Detectar columnas sin variación (todas las filas tienen el mismo valor)
cols_sin_variacion = X_num.columns[X_num.nunique() <= 1].tolist()
print("Columnas sin variación (se eliminan):", len(cols_sin_variacion))

# d) Detectar dummies casi siempre 0 o casi siempre 1
binary_cols = []
for col in X_num.columns:
    valores = set(X_num[col].dropna().unique())
    if valores.issubset({0, 1}):
        binary_cols.append(col)

# Proporción de 1s en cada dummy
proporcion_positivos = X_num[binary_cols].mean()

umbral = 0.01
cols_casi_todo_cero = proporcion_positivos[proporcion_positivos < umbral].index.tolist()
cols_casi_todo_uno = proporcion_positivos[proporcion_positivos > (1 - umbral)].index.tolist()

print("Dummies casi siempre 0 (se eliminan):", len(cols_casi_todo_cero))
print("Dummies casi siempre 1 (se eliminan):", len(cols_casi_todo_uno))

# Lista final de columnas a eliminar por poca información
cols_eliminar_por_poca_info = list(
    set(cols_sin_variacion + cols_casi_todo_cero + cols_casi_todo_uno)
)

# Creamos la matriz final X eliminando esas columnas
X = X_num.drop(columns=cols_eliminar_por_poca_info).copy()

feature_names = X.columns.tolist()

print("Variables finales usadas como features:", len(feature_names))

# Variable objetivo (vector numpy)
y = df[target_col].values.astype("float32")


# ==============================================================
# 4. PARTICIÓN TRAIN / VALIDACIÓN / TEST
# ==============================================================

# Primero hacemos un split train_val vs test (80/20)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.20, random_state=SEED
)

# Luego, dentro de train_val, separamos train y validación (80/20)
# Resultado final aprox: 64% train, 16% val, 20% test
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.20, random_state=SEED
)

print("Tamaño X_train:", X_train.shape)
print("Tamaño X_val:", X_val.shape)
print("Tamaño X_test:", X_test.shape)

# Convertimos a numpy float32 para Keras
X_train_np = X_train.to_numpy().astype("float32")
X_val_np   = X_val.to_numpy().astype("float32")
X_test_np  = X_test.to_numpy().astype("float32")

y_train = y_train.astype("float32")
y_val   = y_val.astype("float32")
y_test  = y_test.astype("float32")


# ==============================================================
# 5. PREPROCESAMIENTO: NORMALIZACIÓN GLOBAL PARA TODAS LAS REDES
# ==============================================================

input_dim = X_train_np.shape[1]
print("\nDimensión de entrada (número de features):", input_dim)

# Creamos UNA capa de normalización, la adaptamos una sola vez
normalizer_global = keras.layers.Normalization(axis=-1)
normalizer_global.adapt(X_train_np)

# Guardamos sus pesos para clonarla en cada modelo sin volver a llamar adapt()
normalizer_weights = normalizer_global.get_weights()


# ==============================================================
# 6. RESUMEN DESCRIPTIVO DEL PRECIO (y)
# ==============================================================

print("\nResumen estadístico del precio (y):")
print(pd.Series(y).describe())

plt.figure(figsize=(8, 4))
plt.hist(y, bins=50)
plt.xlabel("Precio por noche")
plt.ylabel("Frecuencia")
plt.title("Distribución del precio (antes de modelar)")
plt.tight_layout()
plt.show()


# ==============================================================
# 7. DEFINICIÓN DE CONFIGURACIONES DE MODELOS (INSPIRADO EN TALLER 5)
# ==============================================================

def build_model(input_dim, hidden_layers, activation="relu", learning_rate=0.001):
    """
    Construye un modelo de red neuronal sencilla para regresión de precios.
    - input_dim: número de variables de entrada
    - hidden_layers: lista con el número de neuronas por capa oculta, ej. [64] o [64, 32]
    - activation: 'relu' o 'elu'
    """
    model = keras.Sequential(name="nn_regression_price")

    # Capa de entrada
    model.add(keras.layers.Input(shape=(input_dim,)))

    # Capa de normalización: clonamos la global y le ponemos los mismos pesos
    norm_layer = keras.layers.Normalization(axis=-1, name="normalization")
    norm_layer.build((None, input_dim))
    norm_layer.set_weights(normalizer_weights)
    model.add(norm_layer)

    # Capas ocultas
    for i, units in enumerate(hidden_layers, start=1):
        model.add(
            keras.layers.Dense(
                units,
                activation=activation,
                name=f"dense_hidden_{i}"
            )
        )

    # Capa de salida (regresión)
    model.add(keras.layers.Dense(1, activation="linear", name="output"))

    # Compilación: MAE como loss, MAE y MSE como métricas
    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss="mae", metrics=["mae", "mse"])

    return model


# --------------------------------------------------------------
# 7.1 Configuraciones de modelos a probar (búsqueda simple)
# --------------------------------------------------------------
model_configs = [
    {
        "name": "base_relu_64",
        "hidden_layers": [64],
        "activation": "relu",
        "learning_rate": 0.001,
        "batch_size": 128,
    },
    {
        "name": "base_elu_64",
        "hidden_layers": [64],
        "activation": "elu",
        "learning_rate": 0.001,
        "batch_size": 128,
    },
    {
        "name": "deep_relu_64_32",
        "hidden_layers": [64, 32],
        "activation": "relu",
        "learning_rate": 0.001,
        "batch_size": 128,
    },
    {
        "name": "deep_elu_64_32",
        "hidden_layers": [64, 32],
        "activation": "elu",
        "learning_rate": 0.001,
        "batch_size": 128,
    },
]

# --------------------------------------------------------------
# 7.2 Entrenamiento de cada configuración + registro en MLflow
# --------------------------------------------------------------

results = []
best_model = None
best_history = None
best_val_mae = np.inf
best_params = None

for config in model_configs:
    name = config["name"]
    hidden_layers = config["hidden_layers"]
    activation = config["activation"]
    lr = config["learning_rate"]
    batch_size = config["batch_size"]

    print("\n===================================================")
    print(f"Ejecutando modelo: {name}")
    print("---------------------------------------------------")
    print(f"Capas ocultas: {hidden_layers}")
    print(f"Activación:    {activation}")
    print(f"learning_rate: {lr}")
    print(f"batch_size:    {batch_size}")

    model = build_model(
        input_dim=input_dim,
        hidden_layers=hidden_layers,
        activation=activation,
        learning_rate=lr,
    )

    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=15,
            restore_best_weights=True
        )
    ]

    history = model.fit(
        X_train_np, y_train,
        validation_data=(X_val_np, y_val),
        epochs=60,
        batch_size=batch_size,
        verbose=1,
        callbacks=callbacks,
    )

    # Mejor val_mae y val_mse durante el entrenamiento
    val_mae = float(np.min(history.history["val_mae"]))
    val_mse = float(np.min(history.history["val_mse"]))

    # Evaluación en test para comparar entre modelos
    y_pred_test_tmp = model.predict(X_test_np).flatten()
    test_mae_tmp = mean_absolute_error(y_test, y_pred_test_tmp)
    test_mse_tmp = mean_squared_error(y_test, y_pred_test_tmp)
    test_r2_tmp = r2_score(y_test, y_pred_test_tmp)

    # Registro en MLflow (si está disponible)
    if MLFLOW_AVAILABLE:
        with mlflow.start_run(run_name=name):
            mlflow.log_param("hidden_layers", hidden_layers)
            mlflow.log_param("activation", activation)
            mlflow.log_param("learning_rate", lr)
            mlflow.log_param("batch_size", batch_size)

            mlflow.log_metric("val_mae", val_mae)
            mlflow.log_metric("val_mse", val_mse)
            mlflow.log_metric("test_mae", test_mae_tmp)
            mlflow.log_metric("test_mse", test_mse_tmp)
            mlflow.log_metric("test_r2", test_r2_tmp)

            # Opcional: guardar el modelo
            # mlflow_keras.log_model(model, "model")

    # Guardamos resultados en una lista
    results.append({
        "model_name": name,
        "hidden_layers": hidden_layers,
        "activation": activation,
        "learning_rate": lr,
        "batch_size": batch_size,
        "val_mae": val_mae,
        "val_mse": val_mse,
        "test_mae": test_mae_tmp,
        "test_mse": test_mse_tmp,
        "test_r2": test_r2_tmp,
    })

    # Actualizamos el mejor modelo según val_mae
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_history = history
        best_params = {
            "model_name": name,
            "hidden_layers": hidden_layers,
            "activation": activation,
            "learning_rate": lr,
            "batch_size": batch_size,
        }

# --------------------------------------------------------------
# 7.3 Resumen comparativo de modelos
# --------------------------------------------------------------

df_resultados_modelos = pd.DataFrame(results)
df_resultados_modelos = df_resultados_modelos.sort_values("val_mae").reset_index(drop=True)

print("\n===================================================")
print("RESUMEN DE MODELOS (ordenados por val_mae)")
print("===================================================\n")
print(df_resultados_modelos)


# ==============================================================
# 8. EVALUACIÓN DEL MEJOR MODELO EN CONJUNTO DE TEST
# ==============================================================

if best_model is None:
    raise RuntimeError("No se entrenó ningún modelo. Revisa la definición de la búsqueda.")

# Predicciones en el conjunto de prueba (sin restricciones)
y_pred_test_raw = best_model.predict(X_test_np).flatten()

# --------------------------------------------------------------
# Recorte a un mínimo razonable (precio no negativo)
# --------------------------------------------------------------
min_price = 0.0  # si quieres, podrías usar y.min() en vez de 0.0
y_pred_test = np.maximum(y_pred_test_raw, min_price)

# Cálculo de métricas de desempeño usando las predicciones recortadas
test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)
test_rmse = math.sqrt(test_mse)
test_r2 = r2_score(y_test, y_pred_test)

# --------------------------------------------------------------
# 8a. BASELINE: predecir siempre la media del precio de train
# --------------------------------------------------------------
media_train = float(y_train.mean())
baseline_pred = np.full_like(y_test, fill_value=media_train, dtype=float)

baseline_mae = mean_absolute_error(y_test, baseline_pred)
baseline_mse = mean_squared_error(y_test, baseline_pred)

mejora_relativa_mae = (1 - test_mae / baseline_mae) * 100

print("\n====== Comparación con baseline ======")
print(f"Media precio train: {media_train:.4f}")
print(f"MAE baseline (media de train):      {baseline_mae:.4f}")
print(f"MSE baseline (media de train):      {baseline_mse:.4f}")
print(f"MAE red neuronal (mejor modelo):    {test_mae:.4f}")
print(f"MSE red neuronal (mejor modelo):    {test_mse:.4f}")
print(f"Mejora relativa vs baseline (MAE):  {mejora_relativa_mae:.2f}%")
print("======================================\n")


# ==============================================================
# 8b. GRÁFICOS: PRECIO REAL vs PRECIO PREDICHO 
# ==============================================================

df_pred_vs_real = pd.DataFrame({
    "precio_real": y_test,
    "precio_predicho": y_pred_test
}).reset_index(drop=True)

# 8b.1 Serie en el orden original
plt.figure(figsize=(10, 5))
plt.plot(df_pred_vs_real["precio_real"].values, label="Precio real")
plt.plot(df_pred_vs_real["precio_predicho"].values, label="Precio predicho", alpha=0.8)
plt.xlabel("Observación en test")
plt.ylabel("Precio")
plt.title("Precio real vs precio predicho - Conjunto de test (orden original)")
plt.legend()
plt.tight_layout()
plt.show()
# plt.savefig("precio_real_vs_predicho_serie_test.png", dpi=300)

# 8b.3 Scatter real vs predicho con línea ideal
plt.figure(figsize=(6, 6))
plt.scatter(df_pred_vs_real["precio_real"],
            df_pred_vs_real["precio_predicho"],
            s=5, alpha=0.4)
max_price = df_pred_vs_real["precio_real"].max()
plt.plot([0, max_price], [0, max_price], linestyle="--")
plt.xlabel("Precio real")
plt.ylabel("Precio predicho")
plt.title("Dispersión precio real vs predicho (conjunto de test)")
plt.tight_layout()
plt.show()
# plt.savefig("precio_real_vs_predicho_scatter_test.png", dpi=300)


# ==============================================================
# 8d. FUNCIÓN PARA EJEMPLO DE PREDICCIÓN (PREGUNTA 1)
# ==============================================================

def mostrar_prediccion_ejemplo(idx=0):
    """
    Muestra el precio real y predicho para un ejemplo del conjunto de test.
    Esto ayuda a responder:
    'Dado un determinado departamento, ¿cuál será el precio más adecuado por noche?'
    """
    x_sample = X_test_np[idx:idx+1]
    y_real = float(y_test[idx])

    # Predicción cruda del modelo
    y_pred_raw = float(best_model.predict(x_sample).flatten()[0])
    # Recorte a mínimo 0 para mostrar algo con sentido económico
    y_pred = max(y_pred_raw, 0.0)

    print("\nEjemplo de predicción para un departamento del conjunto de test")
    print("----------------------------------------------------------------")
    print(f"Índice interno en X_test: {idx}")
    print(f"Precio real        : {y_real:.2f}")
    print(f"Precio predicho    : {y_pred:.2f}")
    print(f"Predicción (cruda) : {y_pred_raw:.2f}")
    print(f"Error absoluto     : {abs(y_real - y_pred):.2f}")


# Llamada de ejemplo (puedes cambiar el índice):
# mostrar_prediccion_ejemplo(idx=0)


# ==============================================================
# 8e. ANÁLISIS DE CORRELACIONES PARA RECOMENDACIONES (PREGUNTA 2)
# ==============================================================

# Unimos X y y para calcular correlaciones simples con el precio
y_full_series = pd.Series(y, name=target_col)
df_corr = pd.concat(
    [X.reset_index(drop=True), y_full_series.reset_index(drop=True)],
    axis=1
)

corr_with_price = df_corr.corr()[target_col].drop(target_col).sort_values(ascending=False)

print("\n===================================================")
print("CORRELACIONES DE FEATURES CON EL PRECIO")
print("===================================================")

print("\nTop 10 variables con correlación POSITIVA con el precio:")
for feat, val in corr_with_price.head(10).items():
    print(f"  {feat:30s} -> corr = {val:.3f}")

print("\nTop 10 variables con correlación NEGATIVA con el precio:")
for feat, val in corr_with_price.tail(10).items():
    print(f"  {feat:30s} -> corr = {val:.3f}")


def imprimir_recomendaciones_basadas_en_correlaciones(top_k=5):
    """
    Genera recomendaciones textuales a partir de las correlaciones con el precio.
    Esto sirve como base para responder:
    '¿Qué consejos les podemos dar a los anfitriones para incrementar el precio...?'
    """
    print("\n===================================================")
    print("Sugerencias para anfitriones basadas en correlaciones simples")
    print("===================================================\n")

    print(f"Variables más asociadas a precios ALTOS (top {top_k}):")
    for feat, val in corr_with_price.head(top_k).items():
        print(f"- {feat} (corr ≈ {val:.2f}): mantener o reforzar esta característica.")

    print(f"\nVariables más asociadas a precios BAJOS (bottom {top_k}):")
    for feat, val in corr_with_price.tail(top_k).items():
        print(f"- {feat} (corr ≈ {val:.2f}): revisar si es posible mejorarla o compensarla.")

    print("\nEn el informe puedes traducir estas correlaciones en consejos concretos,")
    print("por ejemplo, destacar tipos de propiedad, barrios o características que")
    print("el modelo asocia con precios más altos.\n")

# Llamada de ejemplo:
# imprimir_recomendaciones_basadas_en_correlaciones(top_k=5)


# ==============================================================
# 9. GRÁFICO DE LA ARQUITECTURA DEL MEJOR MODELO
# ==============================================================

try:
    keras.utils.plot_model(
        best_model,
        to_file="arquitectura_mejor_modelo.png",
        show_shapes=True,
        show_layer_names=True,
        expand_nested=True
    )
    print("\nDiagrama de arquitectura guardado en 'arquitectura_mejor_modelo.png'.")
    print("Revisa este archivo PNG para incluirlo en tu reporte.")
except Exception as e:
    print("\nNo se pudo graficar la arquitectura del modelo.")
    print("Posibles causas: falta de graphviz/pydot. Error original:")
    print(e)


# ==============================================================
# 10. RESUMEN FINAL DEL MEJOR MODELO DE REGRESIÓN
# ==============================================================

print("\n\n===================================================")
print("RESUMEN DEL MEJOR MODELO DE REGRESIÓN (RED NEURONAL)")
print("===================================================\n")

print(">> Configuración seleccionada:")
for k, v in best_params.items():
    print(f"   {k}: {v}")

print("\n>> Métricas en VALIDACIÓN (mejor valor durante el entrenamiento):")
print(f"   Mejor val_MAE: {best_val_mae:.4f}")

print("\n>> Métricas en TEST (conjunto hold-out):")
print(f"   MAE  (test): {test_mae:.4f}")
print(f"   MSE  (test): {test_mse:.4f}")
print(f"   RMSE (test): {test_rmse:.4f}")
print(f"   R^2  (test): {test_r2:.4f}")

print("\n>> Comparación con baseline (predecir siempre la media de train):")
print(f"   MAE baseline: {baseline_mae:.4f}")
print(f"   Mejora relativa MAE vs baseline: {mejora_relativa_mae:.2f}%")

# --------------------------------------------------------------
# Información sobre arquitectura: capas, activaciones, etc.
# --------------------------------------------------------------

print("\n>> Arquitectura del modelo (best_model.summary()):\n")
best_model.summary()

# Contamos cuántas capas densas lineales y no lineales hay
num_dense_layers = 0
num_linear_layers = 0
num_nonlinear_layers = 0
activations_info = []

for layer in best_model.layers:
    if isinstance(layer, keras.layers.Dense):
        num_dense_layers += 1
        act = layer.activation.__name__ if hasattr(layer, "activation") else "none"
        activations_info.append((layer.name, act))
        if act == "linear":
            num_linear_layers += 1
        else:
            num_nonlinear_layers += 1

print("\n>> Detalle de capas densas y activaciones:")
for name, act in activations_info:
    print(f"   Capa: {name:20s} | Activación: {act}")

print("\n>> Resumen de capas densas:")
print(f"   Total capas densas: {num_dense_layers}")
print(f"   Capas con activación NO lineal (ReLU/ELU): {num_nonlinear_layers}")
print(f"   Capas con activación lineal (incluye la de salida): {num_linear_layers}")

# --------------------------------------------------------------
# Información sobre variables usadas
# --------------------------------------------------------------

print("\n>> Variables de entrada utilizadas (features):")
print(f"   Número total de variables: {len(feature_names)}")
print("   Algunas de ellas (primeras 30):")
for f in feature_names[:30]:
    print("    -", f)


